## Deep Learning    

### 在风控领域，深度学习算法多使用在特征挖掘方面  

### 运行时选择【**风控专用镜像（TensorFlow 2.6）**】  


## 1. 深度神经网络DNN

In [1]:
# https://keras.io

import tensorflow as tf
from sklearn.metrics import roc_auc_score
from tensorflow.keras import layers, models, callbacks

from utils import data_utils

# 加载数据集
train_x, test_x, train_y, test_y = data_utils.get_x_y_split(transform_method='standard')

# 设置随机数种子
tf.random.set_seed(1)

# 设置早停
callback = callbacks.EarlyStopping(monitor='val_loss', patience=30, mode='min')

# 构建DNN模型结构
model = models.Sequential()
model.add(layers.Flatten(input_shape=(train_x.shape[1], 1)))
model.add(layers.Dense(32, activation=tf.nn.relu))
model.add(layers.Dropout(0.3, seed=1))
model.add(layers.Dense(16, activation=tf.nn.relu))
model.add(layers.Dense(1, activation=tf.nn.sigmoid))
# 显示模型的结构
model.summary()

# 设置模型训练参数
model.compile(optimizer='SGD',
              metrics=[tf.metrics.AUC()],
              loss='binary_crossentropy')
# 模型训练
model.fit(train_x, train_y, validation_data=(test_x, test_y), batch_size=16, epochs=240, callbacks=[callback], verbose=2)

# 效果评估
auc_score = roc_auc_score(train_y, model.predict(train_x))
print("训练集AUC", auc_score)
auc_score = roc_auc_score(test_y, model.predict(test_x))
print("测试集AUC", auc_score)


2023-09-29 13:15:21.539234: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 20)                0         
_________________________________________________________________
dense (Dense)                (None, 32)                672       
_________________________________________________________________
dropout (Dropout)            (None, 32)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 16)                528       
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 17        
Total params: 1,217
Trainable params: 1,217
Non-trainable params: 0
_________________________________________________________________
Epoch 1/240


2023-09-29 13:15:21.763791: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


50/50 - 1s - loss: 0.6823 - auc: 0.5344 - val_loss: 0.6309 - val_auc: 0.5800
Epoch 2/240
50/50 - 0s - loss: 0.6460 - auc: 0.5390 - val_loss: 0.6061 - val_auc: 0.5920
Epoch 3/240
50/50 - 0s - loss: 0.6360 - auc: 0.5312 - val_loss: 0.5912 - val_auc: 0.6107
Epoch 4/240
50/50 - 0s - loss: 0.6203 - auc: 0.5646 - val_loss: 0.5812 - val_auc: 0.6252
Epoch 5/240
50/50 - 0s - loss: 0.6068 - auc: 0.5951 - val_loss: 0.5746 - val_auc: 0.6353
Epoch 6/240
50/50 - 0s - loss: 0.5950 - auc: 0.6233 - val_loss: 0.5677 - val_auc: 0.6440
Epoch 7/240
50/50 - 0s - loss: 0.5890 - auc: 0.6395 - val_loss: 0.5624 - val_auc: 0.6590
Epoch 8/240
50/50 - 0s - loss: 0.5838 - auc: 0.6687 - val_loss: 0.5572 - val_auc: 0.6672
Epoch 9/240
50/50 - 0s - loss: 0.5864 - auc: 0.6489 - val_loss: 0.5533 - val_auc: 0.6742
Epoch 10/240
50/50 - 0s - loss: 0.5719 - auc: 0.6839 - val_loss: 0.5486 - val_auc: 0.6788
Epoch 11/240
50/50 - 0s - loss: 0.5691 - auc: 0.6821 - val_loss: 0.5452 - val_auc: 0.6887
Epoch 12/240
50/50 - 0s - loss:

## 2. 卷积神经网络CNN

In [2]:
import tensorflow as tf
from tensorflow.keras import layers, models, callbacks
from sklearn.metrics import roc_auc_score
from tensorflow.keras import layers, models, callbacks

from utils import data_utils

# 加载数据集
train_x, test_x, train_y, test_y = data_utils.get_x_y_split(transform_method='standard')

# 数据预处理
train_x = train_x.to_numpy().reshape((train_x.shape[0], train_x.shape[1], 1))
test_x = test_x.to_numpy().reshape((test_x.shape[0], test_x.shape[1], 1))
train_y = train_y.values.reshape((train_y.shape[0], 1))
test_y = test_y.values.reshape((test_y.shape[0], 1))

# 设置随机数种子，保证每次运行结果一致
tf.random.set_seed(1)
callback = callbacks.EarlyStopping(monitor='val_loss', patience=30, mode='min')

# 构建CNN模型结构
model = models.Sequential()
model.add(layers.Conv1D(filters=16, kernel_size=4, activation='relu', input_shape=(train_x.shape[1], 1)))
model.add(layers.Conv1D(filters=8, kernel_size=1, activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dropout(0.3, seed=1))
model.add(layers.Dense(16, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))
# 显示模型的结构
model.summary()

# 设置模型训练参数
model.compile(optimizer='SGD',
              metrics=[tf.metrics.AUC()],
              loss='binary_crossentropy')
# 模型训练
model.fit(train_x, train_y, validation_data=(test_x, test_y), batch_size=16, epochs=240, callbacks=[callback], verbose=2)

# 测试集效果评估
auc_score = roc_auc_score(train_y, model.predict(train_x))
print("训练集AUC", auc_score)
auc_score = roc_auc_score(test_y, model.predict(test_x))
print("测试集AUC", auc_score)


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d (Conv1D)              (None, 17, 16)            80        
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 17, 8)             136       
_________________________________________________________________
flatten_1 (Flatten)          (None, 136)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 136)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 16)                2192      
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 17        
Total params: 2,425
Trainable params: 2,425
Non-trainable params: 0
____________________________________________________